20200719 신혜연 업로드 lightGBM+예측템플릿 코드

In [48]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

1. data 불러오기

In [49]:
data = pd.read_csv('C:/Users/USER/Desktop/2020 summer dacon/2020_jeju_credit card/data/final_merge.csv', engine='python')

In [50]:
data.head(5)

,Unnamed: 0,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,...,confirmed,released,deceased,YYMM,festival_count,rain_day,sex_age_population,screen_num,total_tourist,total_sale_by_indst
0,1,강원,건강보조식품 소매업,강원,20s,1,1,13,843100.0,16,...,0,0,0,2019-11-01,0,9.1,100115,28,1.808904e+06,33849.0
1,2,강원,건강보조식품 소매업,강원,20s,1,1,3,139000.0,3,...,0,0,0,2019-06-01,8,10.6,100115,28,1.374767e+06,33849.0
2,3,강원,건강보조식품 소매업,강원,20s,1,1,3,27500.0,3,...,0,0,0,2019-08-01,8,13.9,100115,28,1.603895e+06,33849.0
3,4,강원,건강보조식품 소매업,강원,20s,1,1,3,395500.0,3,...,0,0,0,2019-09-01,11,9.8,100115,28,2.062151e+06,33849.0
4,5,강원,건강보조식품 소매업,강원,20s,1,1,3,427510.0,2,...,29,23,1,2020-03-01,0,7.3,100115,28,NaN,33849.0


2. Feature Engineering & Initial Modeling

In [51]:
# feature, target 설정
df_num = data
train_num = df_num.sample(frac=1, random_state=0)
x = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
y = np.log1p(train_num['AMT'])

In [52]:
x.dtypes

Unnamed: 0               int64
CARD_SIDO_NM            object
STD_CLSS_NM             object
HOM_SIDO_NM             object
AGE                     object
SEX_CTGO_CD              int64
FLC                      int64
month                    int64
year                     int64
confirmed                int64
released                 int64
deceased                 int64
YYMM                    object
festival_count           int64
rain_day               float64
sex_age_population       int64
screen_num               int64
total_tourist          float64
total_sale_by_indst    float64
dtype: object

In [53]:
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'YYMM']
for i in enumerate (cat_features) : 
    ca = i[1] 
    x[ca] = x[ca].astype('category') 
    #Test[ca] = Test[ca].astype('category')


In [54]:
x.dtypes

Unnamed: 0                int64
CARD_SIDO_NM           category
STD_CLSS_NM            category
HOM_SIDO_NM            category
AGE                    category
SEX_CTGO_CD               int64
FLC                       int64
month                     int64
year                      int64
confirmed                 int64
released                  int64
deceased                  int64
YYMM                   category
festival_count            int64
rain_day                float64
sex_age_population        int64
screen_num                int64
total_tourist           float64
total_sale_by_indst     float64
dtype: object

In [55]:
k = int(len(x)*0.9)

In [56]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [57]:
# fix the error "Do not support special JSON characters in feature name."
# from https://www.kaggle.com/c/data-science-bowl-2019/discussion/122021
x_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_train.columns]
x_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_val.columns]

3. Model Tuning & Evaluation

In [59]:
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn import preprocessing

In [29]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [ ]:
model = lgb.train(params,
                  train_ds,
                  1000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

(save the model)
[save the lightGBM model 참고](https://stackoverflow.com/questions/55208734/save-lgbmregressor-model-from-python-lightgbm-package-to-disc/55209076)  
[save the lightGBM model 참고2](https://jaeyung1001.tistory.com/67)

In [3]:
import joblib

In [23]:
# save model
joblib.dump(model, 'lgb.pkl')

# load model
load_model = joblib.load('lgb.pkl')

In [24]:
df_num.columns = df_num.columns.tolist()

In [25]:
df_num.columns 

Index(['Unnamed: 0', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE',
       'SEX_CTGO_CD', 'FLC', 'CSTMR_CNT', 'AMT', 'CNT', 'month', 'year',
       'confirmed', 'released', 'deceased', 'YYMM', 'festival_count',
       'rain_day', 'sex_age_population', 'screen_num', 'total_tourist',
       'total_sale_by_indst'],
      dtype='object')

In [4]:
model = joblib.load('lgb.pkl')

In [5]:
from itertools import product

In [ ]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]
confirmeds    = df_num['confirmed'].unique()
releaseds     = df_num['released'].unique()
deceaseds     = df_num['deceased'].unique()
YYMMs          = df_num['YYMM'].unique()
FLCs          = df_num['FLC'].unique()
festival_counts = df_num['festival_count'].unique()
rain_days  = df_num['rain_day'].unique()
sex_age_populations  = df_num['sex_age_population'].unique()
screen_nums  = df_num['screen_num'].unique()
total_tourists  = df_num['total_tourist'].unique()
total_sale_by_indsts  = df_num['total_sale_by_indst'].unique()

comb_list = [CARD_SIDO_NMs, STD_CLSS_NMs,HOM_SIDO_NMs, AGEs, SEX_CTGO_CDs, FLCs, years, months, confirmeds, releaseds, deceaseds, YYMMs, FLCs, festival_counts, rain_days, sex_age_populations, screen_nums, total_tourists, total_sale_by_indsts]
temp = np.array(list(product(*comb_list)))
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [26]:
temp = pd.DataFrame(data=temp, columns=x.columns)

ValueError: Shape of passed values is (1658860, 8), indices imply (1658860, 19)

In [ ]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
# 제출 파일 만들기
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()

-----------------코드 메모장, 코드 쓰레기통--------------------

In [27]:
# 예측 템플릿 만들기 : 순수 for loops version
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]
confirmeds    = df_num['confirmed'].unique()
releaseds     = df_num['released'].unique()
deceaseds     = df_num['deceased'].unique()
YYMMs          = df_num['YYMM'].unique()
FLCs          = df_num['FLC'].unique()
festival_counts = df_num['festival_count'].unique()
rain_days  = df_num['rain_day'].unique()
sex_age_populations  = df_num['sex_age_population'].unique()
screen_nums  = df_num['screen_num'].unique()
total_tourists  = df_num['total_tourist'].unique()
total_sale_by_indsts  = df_num['total_sale_by_indst'].unique()

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                for confirmed in confirmeds:
                                    for released in releaseds:
                                        for deceased in deceaseds:
                                            for YYMM in YYMMs:
                                                for FLC in FLCs:
                                                    for festival_count in festival_counts:
                                                        for rain_day in rain_days:
                                                            for sex_age_population in sex_age_populations:
                                                                for screen_num in screen_nums:
                                                                    for total_tourist in total_tourists:
                                                                        for total_sale_by_indst in total_sale_by_indsts:
                                                                            temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month, confirmed, released, deceased, YYMM, FLC, festival_count, rain_day, sex_age_population, screen_num, total_tourist, total_sale_by_indst])
                                print("한 번 대충 다 돌렸네요")

                                                                            
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=x.columns)

MemoryError: 

###### 대회 측정 기준인 rmsle 측정 함수

In [30]:
def rmsle(y, pred): 
  log_y = np.log1p(y)
  log_pred = np.log1p(pred)
  squared_error = (log_y - log_pred)**2
  rmsle = np.sqrt(np.mean(squared_error))
  return print('Test Data RMSLE: {0:.3f}'.format(rmsle))
CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, YYMM

######  교차검증 수행함수 ( 아직 미완성 )

In [88]:
def get_best_params_model(model, params):
    if model == xgb:
        lbl = preprocessing.LabelEncoder()
        cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'YYMM']
        for i in enumerate (cat_features) :
            ca = i[1]
            x[ca] = lbl.fit_transform(x[ca].astype(str))
            x_train = x[:k]
            y_train = y[:k]
            x_val = x[k:]
            y_val = y[k:]
    elif model == lgb:
        cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'YYMM']
        for i in enumerate (cat_features) : 
            ca = i[1] 
            x[ca] = x[ca].astype('category') 
            x_train = x[:k]
            y_train = y[:k]
            x_val = x[k:]
            y_val = y[k:]
    cv_model = GridSearchCV(model, param_grid=params, scoring="neg_mean_squared_error", cv = 5)
    cv_model.fit(x_train, y_train)
    print("----", model.__class__.__name__, "----")
    print("GridSearchCV 최적 하이퍼 파라미터 :", cv_model.best_params_)

    rmse = np.sqrt(-1*cv_model.best_score_)
    print("GridSearchCV 최적 평균 RMSE값 :", np.round(rmse, 3))

    eval_pred = cv_model.predict(x_val)
    eval_pred = np.expm1(eval_pred)
    rmsle(y_val, eval_pred)
  
    return cv_model.best_estimator_

In [83]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

from sklearn.ensemble import GradientBoostingRegressor

In [84]:
# catboost 도저히 다운이 안 받아집니다..
from  CatBoostRegressor

SyntaxError: invalid syntax (<ipython-input-84-a1ce2f53b8fb>, line 1)

In [76]:
import catboost 

ModuleNotFoundError: No module named 'catboost'

In [85]:
params = {'n_estimators': [1000, 2000]}

In [86]:
xgb = XGBRegressor(random_state=0)
lgb = LGBMRegressor(random_state=0) 

In [89]:
models = [xgb, lgb] # 모형 하나 추가 함
best_models = []
for model in models:
  new_model = get_best_params_model(model=model, params=params)
  best_models.append(new_model)

KeyboardInterrupt: 

In [ ]:
# 모형 불러오기
xgb_reg = best_models[0]
lgb_reg = best_models[1]

# 예측 데이터 세트
xgb_pred = xgb_reg.predict(x_test)
xgb_pred = np.expm1(xgb_pred)

lgb_pred = lgb_reg.predict(x_test)
lgb_pred = np.expm1(lgb_pred)

pred = np.array([xgb_pred, lgb_pred])
print(pred.shape)

# transpose를 이용해 행과 열의 위치 교환
pred = np.transpose(pred)
print(pred.shape)

In [ ]:
final = xgb_pred*0.60 + lgb_pred*0.40
rmsle(y_val, final)